<a href="https://colab.research.google.com/github/Videothek/batteriespeicher-optimierung/blob/main/Fallstudie_Optimierung_Batteriespeicher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie Optimierung Batteriespeicher


# Abhängigkeiten laden

In [56]:
#Installieren des Solvers
! pip install -q pyscipopt

In [57]:
#Import der packages
import pandas as pd
import requests
from io import BytesIO
from pyscipopt import Model, quicksum

# Prognosewerte berechnen

## Prognosedaten aus der Excel-Datei laden



In [58]:
#Pfad zu den Prognosewerten
url_excel = "https://github.com/Videothek/batteriespeicher-optimierung/raw/refs/heads/main/Preisprognosen.xlsx"
resp = requests.get(url_excel)
xlsx = BytesIO(resp.content)

In [59]:
#Prognosewerte einlesen
prog=pd.read_excel(xlsx, sheet_name=0)

In [60]:
#Prognosewerte ausgeben
prog

,Wahrscheinlichkeit,Stunde,Strompreis
0,0.04,1,66
1,0.04,1,72
2,0.04,1,130
3,0.04,1,69
4,0.04,1,94
...,...,...,...
595,0.04,24,78
596,0.04,24,80
597,0.04,24,90
598,0.04,24,72


## Erwartungswert berechnen

In [61]:
#Spalte der Wahrscheinlichkeiten löschen
prog = prog.drop('Wahrscheinlichkeit', axis=1)

In [62]:
#Erwartungswert für jede Stunde berechnen
prog = prog.groupby('Stunde').mean().reset_index()

In [63]:
#Überschrift der Spalte "Strompreise" ändern
prog = prog.rename(columns={"Strompreis": "Erwartungswert"}, errors="raise")

In [64]:
#Erwartungswerte ausgeben
prog

,Stunde,Erwartungswert
0,1,81.52
1,2,72.24
2,3,68.16
3,4,66.64
4,5,66.80
5,6,71.04
6,7,88.56
7,8,92.68
8,9,80.92
9,10,61.08


# Optimierungsmodell

In [65]:
scip = Model()

## Indexmengen

In [66]:
#Menge der Stunden
H = prog["Stunde"].unique().tolist()

In [67]:
H

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]



---



## Parameter

In [68]:
#Nominelle Speicherkapazität
sn = 40

In [69]:
#Wirkungsgrad Wechselrichter
ww = 0.985

In [70]:
#Round Trip Efficiency (RTE)
r = 0.95

In [71]:
#Wirkungsgrad Laden/Entladen
wl = 1-(1-r)/2

In [72]:
#Gesamtwirkungsgrad
w = ww * wl

In [73]:
#C-Rate
c = 0.5

In [74]:
#Depth-of-Discharge
d = 0.8

In [75]:
#Maximale Anzahl an Vollzyklen
z = 2

In [76]:
#Kosten pro Speicherzyklus
fk = 1500

In [77]:
#Anteil des Speicherstandes zum Ende des Tages
x = 0.5

In [78]:
#Speicherstand zum Ende jedes Tages
e = x * sn

### Prognosepreis zu jeder Stunde ( $p_h$ )

Spaltenüberschrift definieren

In [79]:
h = "Stunde"

Liste mit den Stunden vorbereiten

In [80]:
prog.set_index([h], inplace=True)

Erwartungswert den Stunden zuordnen

In [81]:
p = prog.to_dict("dict")["Erwartungswert"]

Liste zur Überprüfung ausgeben

In [82]:
p

{1: 81.52,
 2: 72.24,
 3: 68.16,
 4: 66.64,
 5: 66.8,
 6: 71.04,
 7: 88.56,
 8: 92.68,
 9: 80.92,
 10: 61.08,
 11: 43.08,
 12: 30.88,
 13: 22.44,
 14: 15.96,
 15: 12.52,
 16: 18.44,
 17: 34.88,
 18: 56.56,
 19: 78.44,
 20: 105.6,
 21: 144.8,
 22: 125.12,
 23: 101.36,
 24: 85.28}



---



## Entscheidungsvariablen

In [83]:
#Speicherstand zu jeder vollen Stunde
s={}
for h in H:
  s[h] = scip.addVar(vtype="C", name=f"{h}")

In [84]:
#Gebotsmenge zu jeder vollen Stunde - Kauf
mk={}
for h in H:
    mk[h] = scip.addVar(vtype="C",lb=0, name=f"{h}")

In [85]:
#Gebotsmenge zu jeder vollen Stunde - Verkauf
mv={}
for h in H:
    mv[h] = scip.addVar(vtype="C",lb=0, name=f"{h}")

In [86]:
#Anzahl der Entscheidungsvariablen
print('Anzahl Entscheidungsvariablen =', len(scip.getVars()))

Anzahl Entscheidungsvariablen = 72


## Zielfunktion

max. $G=\sum mv_{h} * p_{h} - \sum mk_{h} * p_{h} - fk * \frac{\sum mk_{h} * w} {d*s^n}$ $\forall h \in H$

In [87]:
scip.setObjective(quicksum(mv[h]*p[h] for h in H) - quicksum(mk[h]*p[h] for h in H) - fk * (quicksum(mk[h] * w for h in H ) / (d * sn)), sense="maximize")

## Nebenbedingungen

###Berücksichtigung von nomineller Speicherkapazität und Depth-of-Discharge


$s^n * (1-d) \le s_h \le s^n$ $\forall h \in H$


In [88]:
for h in H:
  scip.addCons(sn * (1-d) <= s[h])
  scip.addCons(s[h] <= sn)

###Berücksichtigung der C-Rate bei Aufladung

$w * mk_h \le c * s^n$ $\forall h \in H$

In [89]:
for h in H:
  scip.addCons(w * mk[h] <= c * sn)

###Berücksichtigung der C-Rate bei Entladung

$\frac{1}{w} * mv_h \le c * s^n$ $\forall h \in H$

In [90]:
for h in H:
  scip.addCons((1/w) * mv[h] <= c * sn)

###Berücksichtigung der maximalen Anzahl an Vollzyklen (2 Vollzyklen)


$\sum_h (mk_h * w) \le z * d * s^n$ $\forall h \in H$

In [91]:
scip.addCons(quicksum(mk[h] * w for h in H) <= z * d * sn)

c97

###Berücksichtigung der 20 MW zu Tagesbeginn

$s_1 = mk_1 * w - mv_1 * \frac{1}{w} + e$

In [92]:
scip.addCons(s[1] == mk[1] * w - (mv[1] * (1/w)) + e)

c98

###Am Ende des Tages müssen 20 MW gespeichert sein

$s_{24}=e$

In [93]:
scip.addCons(s[24] == e)

c99

###Speicherstand zum Ende jeder Stunde h

$s_h = s_{h-1} + mk_h * w – mv_h * \frac{1}{w}$ $\forall h \in H$

In [94]:
for h in H:
  if h != 1:
    scip.addCons(s[h] == s[h-1] + mk[h] * w - mv[h] * (1/w))
  else:
    scip.addCons(s[h] == mk[h] * w - (mv[h] * (1/w)) + e)

## Optimierung

In [95]:
scip.optimize()
print(scip.getStatus())

optimal


In [96]:
print('LÖSUNG:')
print('Zielfunktionswert (Gewinn) =', round(scip.getObjVal(), 2))
print("\n")
print("Gebotsmenge für Kauf/Verkauf und Speicherstand zu jeder Stunde")
for h in H:
  print(f"{h}:")
  print('\t', f'Gebotsmenge Kauf =', round(scip.getVal(mk[h]), 2))
  print('\t', f'Gebotsmenge Verkauf =', round(scip.getVal(mv[h]), 2))
  print('\t', f'Speicherstand =', round(scip.getVal(s[h]), 2))

LÖSUNG:
Zielfunktionswert (Gewinn) = 1889.18


Gebotsmenge für Kauf/Verkauf und Speicherstand zu jeder Stunde
1:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 20.0
2:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 20.0
3:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 20.0
4:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 20.0
5:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 20.0
6:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 20.0
7:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 20.0
8:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 11.52
	 Speicherstand = 8.0
9:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 8.0
10:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 8.0
11:
	 Gebotsmenge Kauf = 0.0
	 Gebotsmenge Verkauf = 0.0
	 Speicherstand = 8.0
12:
	 Gebotsmenge Kau